In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
MLP 기반 3단계 이상 탐지 시스템 - ONNX 예측/평가 스크립트

독립적인 3-way classification × 2:
- S1 (Fluctuation): [정상, 황색, 적색] - softmax (합=1)
- S2 (Unbalance):   [정상, 황색, 적색] - softmax (합=1)

3단계 경보:
- 정상 (Normal): 0
- 황색 경보 (Yellow Alert): 1
- 적색 경보 (Red Alert): 2

출력: [S1_정상, S1_황색, S1_적색, S2_정상, S2_황색, S2_적색]
      └───── softmax ─────┘  └───── softmax ─────┘
       (독립적 S1 분류)        (독립적 S2 분류)
"""

import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
import pickle
import argparse

# ONNX Runtime
import onnxruntime as ort

from sklearn.metrics import classification_report, confusion_matrix

# =====================================
# 0. 설정
# =====================================
OUTPUT_DIR_MODELS = "models"
OUTPUT_DIR_RESULTS = "results/mlp_3level"

ONNX_MODEL_PATH = os.path.join(OUTPUT_DIR_MODELS, "mlp_classifier_3level.onnx")
SCALER_PATH = os.path.join(OUTPUT_DIR_MODELS, "scaler_mlp_3level.pkl")
SPLIT_PATH = os.path.join(OUTPUT_DIR_MODELS, "data_split_3level.npz")

# 경보 레벨
ALERT_LEVELS = ['Normal', 'Yellow', 'Red']
ANOMALY_TYPES = ['S1_Fluctuation', 'S2_Unbalance']


# =====================================
# 1. ONNX 모델/스케일러/데이터 로드
# =====================================
def load_onnx_model_scaler_and_split():
    """ONNX 모델, 스케일러, train/test 분할 데이터를 로드"""
    if not os.path.exists(ONNX_MODEL_PATH):
        raise FileNotFoundError(f"ONNX model not found: {ONNX_MODEL_PATH}")
    if not os.path.exists(SCALER_PATH):
        raise FileNotFoundError(f"Scaler not found: {SCALER_PATH}")
    if not os.path.exists(SPLIT_PATH):
        raise FileNotFoundError(f"Data split not found: {SPLIT_PATH}")

    # ONNX Runtime 세션 생성
    ort_session = ort.InferenceSession(ONNX_MODEL_PATH)

    # 스케일러 로드
    with open(SCALER_PATH, "rb") as f:
        scaler = pickle.load(f)

    # train/test 분할 데이터 로드
    split = np.load(SPLIT_PATH)
    X_train = split["X_train"]
    X_test = split["X_test"]
    y_train = split["y_train"]
    y_test = split["y_test"]

    return ort_session, scaler, X_train, X_test, y_train, y_test


# =====================================
# 2. 테스트 세트 평가
# =====================================
def evaluate_on_test_set():
    """테스트 세트에 대한 모델 성능 평가"""
    print("\n" + "=" * 60)
    print("📂 Loading ONNX model / scaler / data split")
    print("=" * 60)

    ort_session, scaler, X_train, X_test, y_train, y_test = load_onnx_model_scaler_and_split()

    print(f"✅ ONNX model loaded: {ONNX_MODEL_PATH}")
    print(f"✅ X_train shape: {X_train.shape}")
    print(f"✅ X_test shape: {X_test.shape}")
    print(f"✅ y_train shape: {y_train.shape}")
    print(f"✅ y_test shape: {y_test.shape}")

    # ONNX Runtime 추론
    input_name = ort_session.get_inputs()[0].name
    X_test_float32 = X_test.astype(np.float32)
    ort_inputs = {input_name: X_test_float32}
    
    y_pred_proba = ort_session.run(None, ort_inputs)[0]  # (N, 6)

    # S1, S2 확률 분리 (독립적 분류)
    s1_probs = y_pred_proba[:, :3]  # (N, 3) - S1 독립 분류
    s2_probs = y_pred_proba[:, 3:]  # (N, 3) - S2 독립 분류

    # 예측 레벨 (각각 독립적으로 argmax)
    s1_pred = np.argmax(s1_probs, axis=1)
    s2_pred = np.argmax(s2_probs, axis=1)

    # 실제 레벨
    s1_true = np.argmax(y_test[:, :3], axis=1)
    s2_true = np.argmax(y_test[:, 3:], axis=1)

    print("\n" + "=" * 60)
    print("📈 Classification Report - S1 (Fluctuation)")
    print("=" * 60)
    print(classification_report(s1_true, s1_pred, target_names=ALERT_LEVELS, digits=4, zero_division=0))

    print("\n" + "=" * 60)
    print("📈 Classification Report - S2 (Unbalance)")
    print("=" * 60)
    print(classification_report(s2_true, s2_pred, target_names=ALERT_LEVELS, digits=4, zero_division=0))

    # Confusion Matrix
    print("\n" + "=" * 60)
    print("📊 Confusion Matrices")
    print("=" * 60)

    cm_s1 = confusion_matrix(s1_true, s1_pred)
    cm_s2 = confusion_matrix(s2_true, s2_pred)

    print("\n[S1 Confusion Matrix]")
    print("              Predicted")
    print("         Normal Yellow   Red")
    for i, level in enumerate(ALERT_LEVELS):
        print(f"{level:8s} {cm_s1[i, 0]:6d} {cm_s1[i, 1]:6d} {cm_s1[i, 2]:6d}")

    print("\n[S2 Confusion Matrix]")
    print("              Predicted")
    print("         Normal Yellow   Red")
    for i, level in enumerate(ALERT_LEVELS):
        print(f"{level:8s} {cm_s2[i, 0]:6d} {cm_s2[i, 1]:6d} {cm_s2[i, 2]:6d}")

    # 전체 정확도
    s1_acc = np.mean(s1_true == s1_pred)
    s2_acc = np.mean(s2_true == s2_pred)
    overall_acc = np.mean((s1_true == s1_pred) & (s2_true == s2_pred))

    print("\n" + "=" * 60)
    print("✅ Accuracy Summary")
    print("=" * 60)
    print(f"  S1 (Fluctuation) accuracy: {s1_acc:.4f}")
    print(f"  S2 (Unbalance) accuracy:   {s2_acc:.4f}")
    print(f"  Overall accuracy (both correct): {overall_acc:.4f}")

    # 메트릭 저장
    os.makedirs(OUTPUT_DIR_RESULTS, exist_ok=True)

    metrics = {
        'model_type': 'MLP Classifier (3-Level Alert System) - ONNX',
        'test_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'data_split': {
            'train_samples': int(len(X_train)),
            'test_samples': int(len(X_test)),
        },
        's1_metrics': {
            'accuracy': float(s1_acc),
            'confusion_matrix': cm_s1.tolist(),
        },
        's2_metrics': {
            'accuracy': float(s2_acc),
            'confusion_matrix': cm_s2.tolist(),
        },
        'overall_accuracy': float(overall_acc),
    }

    metrics_path = os.path.join(OUTPUT_DIR_RESULTS, 'mlp_3level_metrics.json')
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(metrics, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Metrics saved: {metrics_path}")

    # 예측 결과 저장
    pred_dump_path = os.path.join(OUTPUT_DIR_RESULTS, "mlp_3level_test_predictions.npz")
    np.savez(
        pred_dump_path,
        y_test=y_test,
        y_pred_proba=y_pred_proba,
        s1_pred=s1_pred,
        s2_pred=s2_pred,
    )
    print(f"✅ Predictions saved: {pred_dump_path}")


# =====================================
# 3. CSV 파일 예측
# =====================================
def predict_from_csv(csv_path: str, ort_session=None, scaler=None):
    """
    특징 벡터 CSV를 입력받아 3단계 경보 예측을 수행
    
    Args:
        csv_path: window_id, start_time, end_time, feature1, feature2, ... 구조
        ort_session: ONNX Runtime 세션
        scaler: StandardScaler 객체
    
    Returns:
        result_df: 예측 결과가 포함된 DataFrame
    """
    if ort_session is None or scaler is None:
        ort_session, scaler, _, _, _, _ = load_onnx_model_scaler_and_split()

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV file not found: {csv_path}")

    df = pd.read_csv(csv_path)

    if df.shape[1] <= 3:
        raise ValueError("CSV에 feature 컬럼이 없습니다.")

    meta_cols = df.iloc[:, :3].copy()
    X_raw = df.iloc[:, 3:].values

    # 특징 정규화
    X_scaled = scaler.transform(X_raw)

    # ONNX Runtime으로 추론
    input_name = ort_session.get_inputs()[0].name
    X_float32 = X_scaled.astype(np.float32)
    ort_inputs = {input_name: X_float32}
    
    y_proba = ort_session.run(None, ort_inputs)[0]  # (N, 6)

    # S1, S2 확률 분리 (독립적인 3-way classification)
    s1_probs = y_proba[:, :3]  # (N, 3) - S1: [정상, 황색, 적색]
    s2_probs = y_proba[:, 3:]  # (N, 3) - S2: [정상, 황색, 적색]
    # 각 확률 벡터의 합은 1 (softmax로 정규화됨)

    # 결과 DataFrame 구성
    result_df = meta_cols.copy()
    
    # S1 확률
    result_df['s1_prob_normal'] = s1_probs[:, 0]
    result_df['s1_prob_yellow'] = s1_probs[:, 1]
    result_df['s1_prob_red'] = s1_probs[:, 2]
    
    # S2 확률
    result_df['s2_prob_normal'] = s2_probs[:, 0]
    result_df['s2_prob_yellow'] = s2_probs[:, 1]
    result_df['s2_prob_red'] = s2_probs[:, 2]

    # 저장
    os.makedirs(OUTPUT_DIR_RESULTS, exist_ok=True)
    
    filename = os.path.basename(csv_path)
    base, ext = os.path.splitext(filename)
    out_path = os.path.join(OUTPUT_DIR_RESULTS, base + "_predicted_3level" + ext)
    
    result_df.to_csv(out_path, index=False)
    print(f"\n   → Saved with predictions: {out_path}")

    return result_df


# =====================================
# 4. 배치 예측
# =====================================
def predict_batch_csvs(csv_paths: list):
    """여러 CSV 파일에 대해 배치로 예측 수행"""
    print("\n" + "=" * 60)
    print(f"📂 Batch prediction on {len(csv_paths)} CSV files")
    print("=" * 60)

    # 모델과 스케일러를 한 번만 로드
    ort_session, scaler, _, _, _, _ = load_onnx_model_scaler_and_split()

    for csv_path in csv_paths:
        try:
            print(f"\n🔮 Processing: {csv_path}")
            predict_from_csv(csv_path, ort_session=ort_session, scaler=scaler)
        except Exception as e:
            print(f"   ❌ Error processing {csv_path}: {e}")

    print("\n" + "=" * 60)
    print("✅ Batch prediction completed")
    print("=" * 60)


# =====================================
# 5. 통계 요약
# =====================================
def summarize_predictions(csv_path: str):
    """예측 결과 CSV의 통계 요약 출력"""
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Prediction CSV not found: {csv_path}")

    df = pd.read_csv(csv_path)

    print("\n" + "=" * 60)
    print(f"📊 Prediction Summary: {os.path.basename(csv_path)}")
    print("=" * 60)

    # 전체 통계
    print(f"\n[Overall Statistics]")
    print(f"  Total windows: {len(df)}")
    print(f"  Time range: {df['start_time'].min():.1f}s ~ {df['end_time'].max():.1f}s")

    # 확률 통계
    print(f"\n[Probability Statistics]")
    print(f"  S1 (Fluctuation) - 독립적 3-way classification:")
    print(f"    Normal: mean={df['s1_prob_normal'].mean():.4f}, std={df['s1_prob_normal'].std():.4f}")
    print(f"    Yellow: mean={df['s1_prob_yellow'].mean():.4f}, std={df['s1_prob_yellow'].std():.4f}")
    print(f"    Red:    mean={df['s1_prob_red'].mean():.4f}, std={df['s1_prob_red'].std():.4f}")
    print(f"    Sum:    mean={df[['s1_prob_normal', 's1_prob_yellow', 's1_prob_red']].sum(axis=1).mean():.4f} (should be 1.0)")

    print(f"\n  S2 (Unbalance) - 독립적 3-way classification:")
    print(f"    Normal: mean={df['s2_prob_normal'].mean():.4f}, std={df['s2_prob_normal'].std():.4f}")
    print(f"    Yellow: mean={df['s2_prob_yellow'].mean():.4f}, std={df['s2_prob_yellow'].std():.4f}")
    print(f"    Red:    mean={df['s2_prob_red'].mean():.4f}, std={df['s2_prob_red'].std():.4f}")
    print(f"    Sum:    mean={df[['s2_prob_normal', 's2_prob_yellow', 's2_prob_red']].sum(axis=1).mean():.4f} (should be 1.0)")


# =====================================
# 6. main
# =====================================
if __name__ == "__main__":
    import sys

    # ---------------------------------------------------------
    # [Step 1] 모델 성능 평가 (Sanity Check)
    # ---------------------------------------------------------
    """
    # 예측을 수행하기 전에, Test Set에 대해 모델이 정상 작동하는지 확인합니다.
    # 필요 없다면 주석 처리 가능합니다.
    try:
        evaluate_on_test_set()
    except Exception as e:
        print(f"\n[Warning] 모델 평가 중 오류 발생 (파일 누락 등): {e}")
        print("평가를 건너뛰고 예측을 진행합니다.")
    """

    # ---------------------------------------------------------
    # [Step 2] 단일 CSV 파일 예측 실행
    # ---------------------------------------------------------
    # 예측할 대상 파일 경로
    target_csv = 'data/processed/1120_features_S2_yellow.csv'
    
    if os.path.exists(target_csv):
        print("\n" + "=" * 60)
        print("🚀 Single Prediction Execution")
        print("=" * 60)
        
        # 1. 예측 수행
        predict_from_csv(target_csv)
        
        # 2. 결과 파일 경로 추론 (predict_from_csv의 저장 로직과 동일하게 구성)
        filename = os.path.basename(target_csv)
        base, ext = os.path.splitext(filename)
        predicted_csv_path = os.path.join(OUTPUT_DIR_RESULTS, base + "_predicted_3level" + ext)
        
        # 3. 결과 요약 출력
        if os.path.exists(predicted_csv_path):
            summarize_predictions(predicted_csv_path)
        else:
            print(f"❌ 결과 파일을 찾을 수 없습니다: {predicted_csv_path}")
    else:
        print(f"\n[Error] 입력 파일을 찾을 수 없습니다: {target_csv}")

    # ---------------------------------------------------------
    # [Step 3] 배치 예측 (Batch Prediction) - 주석 처리됨
    # ---------------------------------------------------------
    """
    batch_datasets = [
        'data/processed/1124_features.csv',
        'data/processed/1120_features_S1_yellow.csv',
        'data/processed/1120_features_S1.csv',
        'data/processed/1120_features_S2_yellow.csv',
        'data/processed/1120_features_S2.csv'
    ]
    
    # 1. 존재하는 파일만 필터링
    valid_datasets = [f for f in batch_datasets if os.path.exists(f)]
    
    if valid_datasets:
        # 일괄 예측 수행
        predict_batch_csvs(valid_datasets)
        
        # 일괄 요약 출력
        print("\n" + "=" * 60)
        print("📊 Batch Summary Reports")
        print("=" * 60)
        
        for input_csv in valid_datasets:
            # 결과 파일 경로 재구성
            fname = os.path.basename(input_csv)
            fbase, fext = os.path.splitext(fname)
            out_path = os.path.join(OUTPUT_DIR_RESULTS, fbase + "_predicted_3level" + fext)
            
            try:
                summarize_predictions(out_path)
            except Exception as e:
                print(f"Skipping summary for {fname}: {e}")
    else:
        print("\n[Batch] 처리할 유효한 파일이 없습니다.")
    """


🚀 Single Prediction Execution

   → Saved with predictions: results/mlp_3level\1120_features_S2_yellow_predicted_3level.csv

📊 Prediction Summary: 1120_features_S2_yellow_predicted_3level.csv

[Overall Statistics]
  Total windows: 736
  Time range: 0.0s ~ 3685.0s

[Probability Statistics]
  S1 (Fluctuation) - 독립적 3-way classification:
    Normal: mean=1.0000, std=0.0002
    Yellow: mean=0.0000, std=0.0002
    Red:    mean=0.0000, std=0.0000
    Sum:    mean=1.0000 (should be 1.0)

  S2 (Unbalance) - 독립적 3-way classification:
    Normal: mean=0.0000, std=0.0000
    Yellow: mean=0.9830, std=0.1085
    Red:    mean=0.0170, std=0.1085
    Sum:    mean=1.0000 (should be 1.0)
